# Introduction

Today we will demonstrate how you can generate advertizing content for your inventory to boost sales. We'll do this taking advantage of Azure Cosmos DB for Mongo DB vCore's [vector similarity search](https://learn.microsoft.com/azure/cosmos-db/mongodb/vcore/vector-search) functionality. We will use OpenAI embeddings to generate vectors for inventory description which expected to vastly enhance its semantics. The vectors are then stored and indexed in the Mongo vCore Database. During the content generation for the advertisement time we will also vectorize the advertisement topic and find matching inventory itmes. We will then use retrival augmented generation (RAG) by sending to top matches to OpenAI to generate a catchy advertisement.

# Scenario

1. Shoe Retailer who wants to sell more shoes 
2. Wants to run advertisement to capitalize on recent trends
2. Wants to generate advertisement content using inventory items that matches the trend

## Azure OpenAI <a class="anchor" id="azureopenai"></a>

Finally, let's setup our Azure OpenAI resource Currently, access to this service is granted only by application. You can apply for access to Azure OpenAI by completing the form at https://aka.ms/oai/access. Once you have access, complete the following steps:

- Create an Azure OpenAI resource following this quickstart: https://learn.microsoft.com/azure/ai-services/openai/how-to/create-resource?pivots=web-portal
- Deploy a `completions` and `embeddings` model 
    - For more information on `completions`, go here: https://learn.microsoft.com/azure/ai-services/openai/how-to/completions
    - For more information on `embeddings`, go here: https://learn.microsoft.com/azure/ai-services/openai/how-to/embeddings
- Copy the endpoint, key, deployment names for (embeddings model, completions model) into the config.json file.

## Create an Azure Cosmos DB for MongoDB vCore resource<a class="anchor" id="cosmosdb"></a>
Let's start by creating an Azure Cosmos DB for MongoDB vCore Resource following this quick start guide: https://learn.microsoft.com/en-us/azure/cosmos-db/mongodb/vcore/quickstart-portal

Then copy the connection details (server, user, pwd) into the environment file.

# Preliminaries <a class="anchor" id="preliminaries"></a>
First, let's start by installing the packages that we'll need later. 

In [ ]:
! pip install numpy
! pip install openai==1.25.2
! pip install pymongo
! pip install python-dotenv
! pip install azure-core
! pip install azure-cosmos
! pip install tenacity
! pip install gradio

Please use the example.env as a template to provide the necessary keys and endpoints in your own .env file.
Make sure to modify the env_name accordingly. 

In [1]:
import json
import time
import openai

from dotenv import dotenv_values
from openai import AzureOpenAI

# specify the name of the .env file name 
env_name = ".env" # following example.env template change to your own .env file name
config = dotenv_values(env_name)

openai.api_type = config['openai_api_type']
openai.api_key = config['openai_api_key']
openai.api_base = config['openai_api_endpoint']
openai.api_version = config['openai_api_version']

client = AzureOpenAI(
    api_key=openai.api_key,
    api_version=openai.api_version,
    azure_endpoint = openai.api_base
)

# Create embeddings <a class="anchor" id="loaddata"></a>
Here we'll load a sample dataset containing descriptions of Azure services. Then we'll user Azure OpenAI to create vector embeddings from this data.

In [2]:
def generate_embeddings(text):
    '''
    Generate embeddings from string of text.
    This will be used to vectorize data and user input for interactions with Azure OpenAI.
    '''
    response = client.embeddings.create(
        input=text, model="embeddings")
    print(response)
    embeddings = response.data[0].embedding
    time.sleep(0.5) # rest period to avoid rate limiting on AOAI for free tier
    return embeddings

generate_embeddings("Shoes for San Francisco summer")

CreateEmbeddingResponse(data=[Embedding(embedding=[0.013928048312664032, -0.018227633088827133, -0.002117219613865018, -0.028481490910053253, -0.0009161046473309398, 0.006358173675835133, -0.03551717475056648, 0.006937965750694275, 0.019374188035726547, -0.019608711823821068, 0.00758290383964777, -0.00019462134514469653, 0.006964024156332016, -0.008905352093279362, -0.0035178419202566147, -0.018318835645914078, 0.01792796514928341, 0.004856576211750507, 0.028716012835502625, -0.017849789932370186, -0.017458919435739517, 0.004286555573344231, 0.004788173828274012, -0.02371286042034626, -0.00694448035210371, -0.004768630024045706, 0.01878788135945797, -0.011022571474313736, -0.02457277663052082, -0.021706387400627136, 0.027256760746240616, -0.00280124437995255, -0.023139582946896553, -0.0028338171541690826, 0.0006429018685594201, -0.006709957495331764, -0.005426596850156784, 0.0009234335157088935, 0.012117010541260242, -0.010325517505407333, 0.012384106405079365, -0.0068663060665130615, 

[0.013928048312664032,
 -0.018227633088827133,
 -0.002117219613865018,
 -0.028481490910053253,
 -0.0009161046473309398,
 0.006358173675835133,
 -0.03551717475056648,
 0.006937965750694275,
 0.019374188035726547,
 -0.019608711823821068,
 0.00758290383964777,
 -0.00019462134514469653,
 0.006964024156332016,
 -0.008905352093279362,
 -0.0035178419202566147,
 -0.018318835645914078,
 0.01792796514928341,
 0.004856576211750507,
 0.028716012835502625,
 -0.017849789932370186,
 -0.017458919435739517,
 0.004286555573344231,
 0.004788173828274012,
 -0.02371286042034626,
 -0.00694448035210371,
 -0.004768630024045706,
 0.01878788135945797,
 -0.011022571474313736,
 -0.02457277663052082,
 -0.021706387400627136,
 0.027256760746240616,
 -0.00280124437995255,
 -0.023139582946896553,
 -0.0028338171541690826,
 0.0006429018685594201,
 -0.006709957495331764,
 -0.005426596850156784,
 0.0009234335157088935,
 0.012117010541260242,
 -0.010325517505407333,
 0.012384106405079365,
 -0.0068663060665130615,
 -0.00303

# Connect and setup Cosmos DB for MongoDB vCore

## Set up the connection

In [3]:
import pymongo

env_name = ".env"
config = dotenv_values(env_name)

mongo_conn = config['mongo_vcore_connection_string']
mongo_client = pymongo.MongoClient(mongo_conn)

/tmp/ipykernel_22483/480358638.py:7: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  mongo_client = pymongo.MongoClient(mongo_conn)


##  Set up the DB and collection

In [4]:
DATABASE_NAME = "DemoDatabase"
COLLECTION_NAME = "DemoCollection"

mongo_client.drop_database(DATABASE_NAME)
db = mongo_client[DATABASE_NAME]
collection = db[COLLECTION_NAME]

if COLLECTION_NAME not in db.list_collection_names():
    # Creates a unsharded collection that uses the DBs shared throughput
    db.create_collection(COLLECTION_NAME)
    print("Created collection '{}'.\n".format(COLLECTION_NAME))
else:
    print("Using collection: '{}'.\n".format(COLLECTION_NAME))

Created collection 'DemoCollection'.



## Create the vector index

In [6]:
db.command({
  'createIndexes': COLLECTION_NAME,
  'indexes': [
    {
      'name': 'vectorSearchIndex',
      'key': {
        "contentVector": "cosmosSearch"
      },
      'cosmosSearchOptions': {
        'kind': 'vector-ivf',
        'numLists': 1,
        'similarity': 'COS',
        'dimensions': 1536
      }
    }
  ]
});

## Upload data to the collection
A simple `insert_many()` to insert our data in JSON format into the newly created DB and collection.

In [7]:
data_file = open(file="./data/shoes_with_vectors.json", mode="r") 
data = json.load(data_file)
data_file.close()

collection.insert_many(data)

InsertManyResult([ObjectId('6639345d10f2c9fa5e4c002b'), ObjectId('6639345d10f2c9fa5e4c002c'), ObjectId('6639345d10f2c9fa5e4c002d'), ObjectId('6639345d10f2c9fa5e4c002e'), ObjectId('6639345d10f2c9fa5e4c002f'), ObjectId('6639345d10f2c9fa5e4c0030'), ObjectId('6639345d10f2c9fa5e4c0031'), ObjectId('6639345d10f2c9fa5e4c0032'), ObjectId('6639345d10f2c9fa5e4c0033'), ObjectId('6639345d10f2c9fa5e4c0034'), ObjectId('6639345d10f2c9fa5e4c0035'), ObjectId('6639345d10f2c9fa5e4c0036'), ObjectId('6639345d10f2c9fa5e4c0037'), ObjectId('6639345d10f2c9fa5e4c0038'), ObjectId('6639345d10f2c9fa5e4c0039'), ObjectId('6639345d10f2c9fa5e4c003a'), ObjectId('6639345d10f2c9fa5e4c003b'), ObjectId('6639345d10f2c9fa5e4c003c'), ObjectId('6639345d10f2c9fa5e4c003d'), ObjectId('6639345d10f2c9fa5e4c003e'), ObjectId('6639345d10f2c9fa5e4c003f'), ObjectId('6639345d10f2c9fa5e4c0040'), ObjectId('6639345d10f2c9fa5e4c0041'), ObjectId('6639345d10f2c9fa5e4c0042'), ObjectId('6639345d10f2c9fa5e4c0043'), ObjectId('6639345d10f2c9fa5e4c00

# Vector Search in Cosmos DB for MongoDB vCore

In [8]:
# Function to assist with vector search
def vector_search(query, num_results=3):
    
    query_vector = generate_embeddings(query)

    embeddings_list = []
    pipeline = [
        {
            '$search': {
                "cosmosSearch": {
                    "vector": query_vector,
                    "numLists": 1,
                    "path": "contentVector",
                    "k": num_results
                },
                "returnStoredSource": True }},
        {'$project': { 'similarityScore': { '$meta': 'searchScore' }, 'document' : '$$ROOT' } }
    ]
    results = collection.aggregate(pipeline)
    return results

## Perform vector search query

In [12]:
query = "Shoes for Zurich Lake in Switzerland weather"
#query = "Footwear for the beach in Geneva plage"

results = vector_search(query, 5)

print("\nResults:\n")
for result in results: 
    print(f"Similarity Score: {result['similarityScore']}")  
    print(f"Title: {result['document']['name']}")  
    print(f"Price: {result['document']['price']}")  
    print(f"Material: {result['document']['material']}") 
    print(f"Image: {result['document']['img_url']}\n") 

CreateEmbeddingResponse(data=[Embedding(embedding=[0.009293043054640293, -0.017612852156162262, 0.005035139620304108, -0.022965645417571068, 0.0044809365645051, 0.031413864344358444, -0.02925112098455429, -0.02822381816804409, 0.01661258190870285, -0.04460660740733147, 0.0006893748650327325, 0.01138820219784975, -0.010678551159799099, -0.03246820345520973, -0.00903621781617403, -0.0011540270643308759, 0.013490119948983192, 0.015950242057442665, 0.03838871791958809, -0.020708279684185982, -0.0035381149500608444, 0.022451993077993393, 0.03687479719519615, -0.018842913210392, -0.004589073825627565, -0.012057301588356495, 0.013632049784064293, -0.019288979470729828, -0.007657467853277922, -0.02616921067237854, 0.015301418490707874, -0.008583392947912216, -0.031116489320993423, -0.005930651444941759, 0.0022353993263095617, -0.005971202626824379, -0.007853466086089611, -0.008144085295498371, 0.008211671374738216, -0.004538384266197681, 0.019045669585466385, -0.017883194610476494, 0.006096236

# Generating Ad content with GPT-4 and DALL.E 3

Finally, we put it all together by creating an ad caption and an ad image via the `Completions` API and `DALL.E 3` API, and combine that with the vector search results.

In [10]:
from openai import AzureOpenAI

def generate_ad_title(ad_topic):
    system_prompt = '''
    You are an intelligent assistant for generating witty and cativating tagline for online advertisement.
        - The ad campaign taglines that you generate are short and typically under 100 characters.
    '''

    user_prompt = f'''Generate a catchy, witty, and short sentence (less than 100 characters) 
                    for an advertisement for selling shoes for {ad_topic}'''
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]

    response = client.chat.completions.create(
        model="completions",
        messages=messages
    )
    
    return response.choices[0].message.content

def Generate_ad_image(ad_topic):
    image_prompt = f'''
        Generate a photorealistic image of an ad campaign for selling {ad_topic}. 
        The image should be clean, with the item being sold in the foreground with an easily identifiable landmark of the city in the background.
        The image should also try to depict the weather of the location for the time of the year mentioned.
        The image should not have any generated text overlay.
    '''

    response = client.images.generate(
        model="dall-e-3",
        prompt= image_prompt,
        size="1024x1024",
        quality="standard",
        n=1,
        )

    return response.data[0].url

def render_html_page(ad_topic):

    # Find the matching shoes from the inventory
    results = vector_search(ad_topic, 4)

    ad_header = generate_ad_title(ad_topic)
    ad_image_url = Generate_ad_image(ad_topic)


    with open('./data/ad-start.html', 'r', encoding='utf-8') as html_file:
        html_content = html_file.read()

    html_content += f'''<header>
            <h2>{ad_header}</h1>
        </header>'''    

    html_content += f'''
            <section class="ad">
            <img src="{ad_image_url}" alt="Base Ad Image" class="ad-image">
        </section>'''

    for result in results: 
        html_content += f''' 
        <section class="product">
            <img src="{result['document']['img_url']}" alt="{result['document']['name']}" class="product-image">
            <div class="product-details">
                <h3 class="product-title" color="gray">{result['document']['name']}</h2>
                <p class="product-price">{"$"+str(result['document']['price'])}</p>
                <p class="product-description">{result['document']['description']}</p>
                <a href="#" class="buy-now-button">Buy Now</a>
            </div>
        </section>
        '''

    html_content += '''</article>
                    </body>
                    </html>'''

    return html_content

# Putting it all together

In [11]:
import gradio as gr

css = """
    button { background-color: purple; color: read; }
    <style>
    </style>
"""

with gr.Blocks(css=css, theme=gr.themes.Default(spacing_size=gr.themes.sizes.spacing_sm, radius_size="none")) as demo:
    subject = gr.Textbox(placeholder="subject", label="Ad keywords")
    btn = gr.Button("Generate Ad")
    output_html = gr.HTML(label="Generated Ad HTML")

    btn.click(render_html_page, [subject], output_html)

    btn = gr.Button("Copy HTML")

if __name__ == "__main__":
    demo.launch()   

/home/dobroegl/miniconda3/envs/ipykernel/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
